# Collect 16S sequences from NCBI

In [3]:
# https://biopython.org/docs/latest/Tutorial/chapter_entrez.html#sec-entrez-webenv
# Börja med att importera rätt paket, och lägg in mejladress för att NCBI ska veta vem man är (idk)
# Kräver att Biopython är installerat

import pandas as pd
from Bio import Entrez
from Bio import SeqIO

In [5]:
# Läsa in vilka arter som vi ska hämta sekvenser för
organism_dataset = pd.read_csv('Data/Given_organisms_species_genus.txt', names=['Organisms'])
organisms = [a for a in organism_dataset.iloc[:, 0]]

In [6]:
# Sök igenom NCBI och hitta 1) fastafiler av 16S genen och 2) taxonomin för arterna som använts
# Sökningen sker med följande filter: 
# - Species = Bacteria
# - Molecule types = Genomic DNA/RNA
# - Source databases = RefSeq
# - Sequence length = [1300, 1800]

# The output files
# - output_fasta: Fastasekvenserna för 16S rRNA-genen
# - output_taxonomy: Taxonomin på nivåerna kingdom/phylum/class/order/family/genus/species för arterna vi hämtar fastasekvenser från
# - not_found: En lista med de arter som inte finns i databasen

output_fasta = open('All_taxa_species_genus.fasta', 'w')
output_taxonomy = open('Taxonomy_species_genus.txt', 'w')
output_taxonomy.write(f'ID, Kingdom, Phylum, Class, Order, Family, Genus, Species\n')
output_notfound = open('Not_found_species_genus.txt', 'w')

# Antal sekvenser att hämta per art
max_sequences = 1

# Loopa genom lista med arter
# För varje art, sök i NCBI enligt kriterierna ovan, och spara resultaten
# Ladda sedan ner det angivna antalet sekvenser och spara i output_fasta, samt organismens taxonomi i output_taxonomy
# Om arten inte finns, skriv det som output och lägg till den i listan för not found
for organism in organisms:
    Entrez.email = 'clara.nordquist.1217@student.uu.se'

    search_term = f'''
    ("{organism}"[Organism] AND 16S[All Fields] AND refseq[filter] AND 
    "1300"[SLEN] : "1800"[SLEN] AND bacteria[filter]) 
    '''
    
    # Först, sök igenom databasen
    stream = Entrez.esearch(db = 'nucleotide', term = search_term, usehistory = 'y', idtype = 'acc')
    search_results = Entrez.read(stream)
    stream.close()
    acc_list = search_results['IdList']
    webenv = search_results['WebEnv']
    query_key = search_results['QueryKey']
    
    # Ladda ner gensekvensen och spara i filen fasta_output
    stream = Entrez.efetch(
    db = 'nucleotide', rettype = 'fasta', retmode = 'text', retmax = max_sequences, webenv = webenv, query_key = query_key, idtype = 'acc')
    data = stream.read()
    
    # Om organismen finns
    if type(data) == str:
        output_fasta.write(data)

        # Ladda ner taxonomin
        stream = Entrez.efetch(db = 'nucleotide', rettype = 'gb', retmode = 'text', retmax = max_sequences, webenv = webenv, query_key = query_key, idtype = 'acc')
        for record in SeqIO.parse(stream, 'genbank'):
            output_taxonomy.write(f'{record.id}, ')
            output_taxonomy.write(f'{str(record.annotations["taxonomy"]).replace("[", "").replace("]", "")}\n')
    
    # Om organismen inte finns
    else:
        print(f'Warning: {organism} not found in database')
        output_notfound.write(f'{organism}\n')

output_fasta.close()
output_taxonomy.close()
output_notfound.close()

## Ladda ned 16S rRNA-sekvenserna

In [4]:
# # Definiera hur många sekvenser som ska hämtas för varje art, samt namnet på outputfilen
# output = open('All_given_species_genus.fasta', 'w')

# # En lista för alla arter som ej finns i databasen
# not_found = []

# # För varje organism i listan, sök i NCBI (RefSeq) efter 16S rRNA-gener med längd mellan 1400 och 1650 bp
# # Spara de [max_sequences] första träffarna i fastaformat, i den angivna outputfilen
# # Om organismens ej finns i databasen, lägg namnen i en lista
# for organism in organisms: 
#     search_term = f'''
#     ({organism}[Organism] AND 16S[All Fields] 
#     AND bacteria[filter] 
#     AND (refseq[filter] AND "1400"[SLEN] : "1650"[SLEN])
#     '''
#     stream = Entrez.esearch(db = 'nucleotide', term = search_term, usehistory = 'y', idtype = 'acc')
#     search_results = Entrez.read(stream)
#     stream.close()
#     acc_list = search_results['IdList']
#     webenv = search_results['WebEnv']
#     query_key = search_results['QueryKey']
    
#     stream = Entrez.efetch(
#     db = 'nucleotide', rettype = 'fasta', retmode = 'text', retmax = 2, webenv = webenv, query_key = query_key, idtype = 'acc')
#     data = stream.read()
    
#     if type(data) == str:
#         output.write(data)
#     else:
#         not_found.append(organism)
# output.close()